In [1]:
%%time
import numpy as np
import matplotlib.pyplot as plt
import sys
import math
import numpy as np


for mod in ["sutton_functions"]:
    if mod in sys.modules:
            del sys.modules[mod]
            
from sutton_functions import *

CPU times: user 1.58 s, sys: 708 ms, total: 2.29 s
Wall time: 444 ms


In [2]:
def integrate_H2O_implicit(n, m, dx, dz, A, B, C, Qup, Q_s, Q_a):
    """
    This function computes the coefficients
      of the second-order system of variable s(t+dt)
      formed by the (known) profile values at s(t).
      
      The system is AA1 d2s/dz2 + AA2 ds/dz + AA3 s=AA4
      The coefficients in AA4 are used from s(t)
      The coefficients in AA1 are mainly the diffusion
      The coefficients in AA2 include advection
      The coefficients in AA3 include advection + s(t+dt)  
    """
    # Setup the tridiagonal solver for mean H2O concentration
    AA1 = - A * B
    AA2 = - C * B
    AA3 = 1 / dx
    AA4 = Qup / dx
    
    upd = (AA1 / (dz ** 2) + AA2 / (2 * dz))
    dia = (-2 * AA1 / (dz ** 2) + AA3)
    lod = (AA1 / (dz ** 2) - AA2 / (2 * dz))
    
    co = np.zeros(m)
    co[:] = AA4

    # Ensure the boundary conditions are state, not flux
    lod[0] = 0
    lod[m-1] = 0
    dia[0] = 1
    dia[m-1] = 1
    upd[0] = 0
    upd[m-1] = 0

    # Enforce surface and upper H2O concentration

    co[0] = Q_s
    co[m-1] = Q_a

    # Call the tridiagonal solver
    Q1 = thomas(lod, dia, upd, co)

    dQdz = our_central_difference(Q1, dz)

    Fq = - A * dQdz
    
    return Q1, Fq

def vapor_concentration_RH( T, RH = 100):
    """
    Calculate the vapor concentration (in g/m³) for a given relative humidity (RH) and temperature (T in °C).
    
    Parameters:
    - RH (float): Relative humidity as a fraction (e.g., 0.5 for 50%).
    - T (float): Temperature in degrees Celsius.
    
    Returns:
    - float: Vapor concentration in g/m³.
    """
    # Constants
    M_w = 18.015  # molar mass of water vapor in g/mol
    R = 8.314  # universal gas constant in J/(mol*K)
    
    # Calculate the saturation vapor pressure (Pa)
    e_s = 6.1094 * math.exp((17.625 * T) / (T + 243.04))*100

    # Actual vapor pressure (Pa)
    e = RH/100 * e_s
    
    # Vapor concentration (g/m³)
    C = (e * M_w) / (R * (T + 273.15))
    
    return C


def vapor_concentration(es, T):
    # Constants
    M = 18.015  # Molar mass of water in g/mol
    R = 8.314  # Universal gas constant in J/(mol·K)
    
    # Water vapor concentration calculation
    rho = (es * M) / (R * T)
    return rho


In [3]:
def get_params(fallow_fraction = 0., fallow_length = 1000, T_c = 30, T_f = 50):
    
    zom = 0.22
    # Flat parameter dictionary
    params = {
        "k": 0.4,            # von Karman constant        
        "zom": zom,         # Momentum roughness length (m)

        "Lx": 1000,           # Domain length in x-direction
        "Hmax": 100,         # Maximum height

        "fallow_fraction": fallow_fraction,
        "fallow_length" : fallow_length,
        "dz": 0.5,
        "dx": 5,
        
        'T_c' : T_c,  # Cultivated temperature
        'T_f' : T_f,  # Fallow temperature

    }
    Ubar_10 = 4
    ustar = 0.4*Ubar_10/np.log(10/zom)    # Friction velocity (m/s)
    print (ustar)
    params['xmin'] = 0
    params['ustar'] = ustar
    params['xmax'] = params['Lx']
    params['zmin'] = params['zom']
    params['zmax'] = params['Hmax']
    params['field_size'] = int(params['fallow_length']/params['dx'])
    
    if fallow_fraction > 0:
        params['fallow_size'] = int(params['field_size']*(1- fallow_fraction)/fallow_fraction)
    else:
        params['fallow_size'] = 0
    
    z = np.arange(params['zmin'] + params['dz'], params['zmax'] + params['dz'], params['dz'])
    nz = len(z)
    x = np.arange(params['xmin'], params['xmax'] + params['dx'], params['dx'])
    nx = len(x)   

    RH_c = 42
    es_c = saturation_vapor_pressure(T_c + 273.15)
    Q_c = vapor_concentration_RH(T_c, RH_c)
    Q_s = Q_c

    print(f"Cultivated \nSaturation vapor pressure at {T_c}°C is {es_c:.2f} Pa")
    print(f"Water vapor concentration at {T_c}°C is {Q_c:.2f} g/m³")

    
    RH_f = 12    
    es_f = saturation_vapor_pressure(T_f + 273.15)
    Q_f = vapor_concentration_RH(T_f, RH_f)
    Q_a  = vapor_concentration_RH(T_f, 11.5)

    print(f"Fallow \nSaturation vapor pressure at {T_f}°C is {es_f:.2f} Pa")
    print(f"Water vapor concentration at {T_f}°C is {Q_f:.2f} g/m³")

    print(f"Atmosphere ")
    print(f"Water vapor concentration is {Q_a:.2f} g/m³")

    params['Q_c'] =   Q_c   # Surface water vapor concentration for vegetated areas
    params['Q_f'] =   Q_f   # Surface water vapor concentration for fallowed areas        
    params['Q_a'] =   Q_a   # Upwind background atmospheric water vapor concentration for fallow areas
    params['nx'] = nx
    params['nz'] =  nz    
            
    return params


In [4]:

def populate_Qc_array(params):
    # Retrieve parameters from the dictionary
    x = np.arange(params['xmin'], params['xmax'] + params['dx'], params['dx'])
    nx = len(x)
    field_size = params["field_size"]
    gap_size = params["gap_size"]
    Q_c = params["Q_c"]
    Q_f = params["Q_f"]
    
    # Initialize the array with zeros or any other default value
    Q_c_array = [0] * (nx)
    # Populate the array with alternating fieldes
    i = 0
    while i < nx :
        # Assign a vegetated field (Q_c) of fixed length
        Q_c_array[i:i + field_size+1] = [Q_c] * min(field_size, nx  - i)
        i += field_size
        
        # Assign a fallow field (Qf) of fixed length, if within bounds

        Q_c_array[i:i + gap_size] = [Q_f] * min(gap_size, nx  - i)
        i += gap_size
            
    return np.array(Q_c_array)



In [5]:
params = get_params()

# Call the function with the parameter dictionary
Qs_array = populate_Qc_array(params)

ustar = params['ustar']
k = params['k']
Hmax = params['Hmax']
zom = params['zom']
Q_a = params['Q_a']
Q_c = params['Q_c']
Q_f = params['Q_f']
z = np.arange(params['zmin'] + params['dz'], params['zmax'] + params['dz'], params['dz'])
dz = params['dz']
dx = params['dx']

nx = params['nx']
nz = params['nz']

# check these equations
LE_c = ustar*k*(Q_c - Q_a)/np.log(Hmax/zom)  # m/s g/m3 = g /m2 s
LE_f = ustar*k*(Q_f - Q_a)/np.log(Hmax/zom)

# Specify upwind wv concentration (as background)
Qup_f = Q_f - LE_f/(k*ustar)*np.log(z/zom)
# Qup = np.ones_like(z) * Q_a

# Generate the mean velocity profile from the log-law
U = (ustar / k) * np.log(z / zom)

0.4192089038657208
Cultivated 
Saturation vapor pressure at 30°C is 4353.80 Pa
Water vapor concentration at 30°C is 12.72 g/m³
Fallow 
Saturation vapor pressure at 50°C is 13163.46 Pa
Water vapor concentration at 50°C is 9.95 g/m³
Atmosphere 
Water vapor concentration is 9.53 g/m³


KeyError: 'gap_size'

In [ ]:
def varying_Qs(nx, nz, dx, dz, k, z, ustar, U, Qup_f, Qs_array, Q_a, h):
    
    # Setup coefficients for the implicit scheme
    zom = h/10
    d = 2/3*h
    # Setup coefficients for the implicit scheme
    lm = k * (z - d)    # mixing length
    lm[z < h] = k*h/3
    
    A = lm * ustar
    B = 1.0 / U
    C = our_central_difference(A, dz)

    # Initialize upwind water vapor concentrations and fluxes
    Q1 = Qup_f
    Q = np.zeros((nx, nz))
    Q[0, :] = Q1
    FluxQ = np.zeros((nx, nz))

    # Perform downwind calculations by marching along x
    for i, Qs_i in enumerate(Qs_array):

        Q2, Fq = integrate_H2O_implicit(nx, nz, dx, dz, A, B, C, Q1, Qs_i, Q_a)
        Q[i , :] = Q2
        FluxQ[i , :] = Fq

        Q1 = Q2

    return Q, FluxQ

def uniform_cultivated(nx, nz, dx, dz, k, z, ustar, U, Qup_f, Q_c, Q_a, h):

    zom = h/10
    d = 2/3*h
    # Setup coefficients for the implicit scheme
    lm = k * (z - d)    # mixing length
    lm[z < h] = k*h/3
    
    A = lm * ustar
    B = 1.0 / U
    C = our_central_difference(A, dz)

    # Upwind wv concentrations and fluxes
    Q1 = Qup_f
    Q_uniform = np.zeros((nx , nz))
    Q_uniform[0, :] = Q1
    FluxQ_uniform = np.zeros((nx , nz))

    # Begin downwind calculations by marching along x
    for i in range(nx):
        Q2, Fq = integrate_H2O_implicit(nx, nz, dx, dz, A, B, C, Q1, Q_c, Q_a)
        Q_uniform[i, :] = Q2
        FluxQ_uniform[i, :] = Fq
        Q1 = Q2

    return Q_uniform, FluxQ_uniform

def uniform_fallow(nx, nz, dx, dz, k, z, ustar, U, Qup_f, Q_f, Q_a, h):
    
    zom = h/10
    d = 2/3*h
    # Setup coefficients for the implicit scheme
    lm = k * (z - d)    # mixing length
    lm[z < h] = k*h/3
    
    A = lm * ustar
    B = 1.0 / U
    C = our_central_difference(A, dz)    
    
    # Upwind wv concentrations and fluxes
    Q1 = Qup_f
    Q_fallow = np.zeros((nx , nz))
    Q_fallow[0, :] = Q1
    FluxQ_fallow = np.zeros((nx , nz))
    
    # Begin downwind calculations by marching along x
    for i in range(nx):
        Q2, Fq = integrate_H2O_implicit(nx, nz, dx, dz, A, B, C, Q1, Q_f, Q_a)

        Q_fallow[i, :] = Q2
        FluxQ_fallow[i, :] = Fq
        Q1 = Q2

    return Q_fallow, FluxQ_fallow

def ET_enhance(FluxQ_uniform, FluxQ_fallow, FluxQ):
    
    continuous_cultivation = FluxQ_uniform[nx-1, 0]*dx
    
    uniform_flux = FluxQ_uniform[nx-1, 0]*(Qs_array > Q_f).mean() + \
                FluxQ_fallow[nx-1, 0]*(Qs_array == Q_f).mean()
    uniform_flux = uniform_flux*dx
    
    FluxQ_force = FluxQ.copy()
    FluxQ_force[Qs_array < Q_c] = FluxQ_fallow[-1,0]
    
    return  continuous_cultivation, uniform_flux, FluxQ_force[:, 0].mean()*dx


In [ ]:
Q_c

In [ ]:
h = 2.3
irg_ind = 2
Q, FluxQ = varying_Qs(nx, nz, dx, dz, k, z, ustar, U, Qup_f, Qs_array, Q_a, h)
    
Q_uniform, FluxQ_uniform = uniform_cultivated(nx, nz, dx, dz, k, z, ustar, U, Qup_f, Q_c, Q_a, h)
Q_fallow, FluxQ_fallow = uniform_fallow(nx, nz, dx, dz, k, z, ustar, U, Qup_f, Q_f, Q_a, h)        
    
continuous_cultivation, uniform_flux, patchy_flux = ET_enhance(FluxQ_uniform, FluxQ_fallow, FluxQ)    
FluxQ[Qs_array < Q_c] = FluxQ_fallow[-1,0]



In [ ]:
# plt.plot(FluxQ_fallow[:, 0]*2260)
plt.plot(FluxQ_fallow[:, irg_ind]*2260)

In [ ]:
# Plot excess water vapor and surface flux
plt.figure(figsize = (8, 6))
x = np.arange(params['xmin'], params['xmax'] + params['dx'], params['dx'])
QQ = np.sum(Q - Q_a, axis=1) * dz
QQ_uniform = np.sum(Q_uniform - Q_a, axis=1) * dz

plt.subplot(2, 1, 1)
plt.plot(x, QQ, '-', label = 'Fallow patches')
plt.xlabel('x (m)', fontsize=12, fontweight='normal')
plt.ylabel(r'$\int [q(x,z) - q_a]dz$', fontsize=12, fontweight='normal')
plt.title('Excess water vapor (g m$^{-2}$)', fontsize=12)

plt.subplot(2, 1, 2)
plt.plot(x[2:], FluxQ[2:, irg_ind], '-')
plt.xlabel('x (m)', fontsize=12, fontweight='normal')
plt.ylabel('ET (g m$^{-2}$ s$^{-1}$)', fontsize=12, fontweight='normal')
plt.title('Surface Flux (g m$^{-2}$ s$^{-1}$)', fontsize=12)
plt.tight_layout()

In [ ]:
plt.figure(figsize = (8, 3))
ax = plt.gca()

plt.plot(x, np.ones(nx ) * FluxQ_uniform[nx-1, irg_ind], '--',  
         label = "Uniform cultivation, \n  mean ET = {0:.2f} g m$^2$/s".format(
             FluxQ_uniform[nx-1, irg_ind]*dx))


uniform_flux = FluxQ_uniform[nx-1, irg_ind]*(Qs_array > Q_f).mean() + \
                FluxQ_fallow[nx-1, irg_ind]*(Qs_array == Q_f).mean()

plt.plot(x[2:], FluxQ[2:, irg_ind], '-', label = 
    "Including advection,  \n mean ET = {0:.2f} g m$^2$/s".format(FluxQ[2:, irg_ind].mean()*dx))

plt.xlabel('x (m)', fontsize=12, fontweight='normal')
plt.ylabel('ET (g m$^{-2}$ s$^{-1}$)', fontsize=12, fontweight='normal')
plt.title('Surface Flux (g m$^{-2}$ s$^{-1}$)', fontsize=12)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))


In [ ]:

plt.figure(figsize = (8, 3))
ax = plt.gca()

plt.plot(x, np.ones(nx ) * FluxQ_uniform[nx-1, irg_ind], '--',  
         label = "Continuous cultivation, \n  mean ET = {0:.2f} g m$^2$/s".format(
             FluxQ_uniform[nx-1, irg_ind]))


# FluxQ[Qs_array < Q_c] = FluxQ_fallow[-1,0]
plt.plot(x[:], FluxQ[:, irg_ind], '-', label = "Including advection,  \n mean ET = {0:.2f} g m$^2$/s".format(
    FluxQ[:, irg_ind].mean()))

plt.xlabel('x (m)', fontsize=12, fontweight='normal')
plt.ylabel('ET (gm m$^{-2}$ s$^{-1}$)', fontsize=12, fontweight='normal')
plt.title('Surface Flux (gm m$^{-2}$ s$^{-1}$)', fontsize=12)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# (FluxQ[:, 0].sum()*dx/nx - FluxQ[nx:,  0].sum()*dx)/(FluxQ[nx:,  0].sum()*dx)

In [ ]:
plt.figure(figsize = (8, 3))
ax = plt.gca()

plt.plot(x, np.ones(nx) * FluxQ_uniform[nx-1, irg_ind]*2260, '-',  
         label = "Continuous cultivation, \n  mean ET = {0:.0f} W m$^-2$".format(
             FluxQ_uniform[nx-1, irg_ind]*2260))


plt.plot(x[2:], FluxQ[2:, irg_ind]*2260, ':', 
         label = "Including advection,  \n mean ET = {0:.0f} W m$^-2$".format(
             FluxQ[2:, irg_ind].mean()*2260))

plt.plot(-x[10:], FluxQ_fallow[10::, irg_ind]*2260, ':', c  = 'C1')


combined = np.concatenate([FluxQ_fallow[10::, irg_ind], FluxQ[:2, irg_ind]])*2260

plt.xlabel('x (m)', fontsize=12, fontweight='normal')
plt.ylabel('ET (W m$^-2$)', fontsize=12, fontweight='normal')
plt.title('Surface Flux (W m$^-2$)', fontsize=12)
ax.legend()#loc='center left',bbox_to_anchor=(1, 0.5))

# (FluxQ[2:, irg_ind].sum()*dx/nx - FluxQ[nx:,  irg_ind].sum()*dx)/(FluxQ[nx:,  irg_ind].sum()*dx)


In [ ]:
plt.figure(figsize = (8, 3))
ax = plt.gca()

# plt.plot(x, np.ones(nx) * FluxQ_uniform[nx-1, irg_ind]*2260, '-',  
#          label = "Continuous cultivation, \n  mean ET = {0:.0f} W m$^-2$".format(
#              FluxQ_uniform[nx-1, irg_ind]*2260))

# plt.plot(x[2:], FluxQ[2:, irg_ind]*2260, ':', c  = 'C1',
#          label = "Including advection,  \n mean ET = {0:.0f} W m$^-2$".format(
#              FluxQ[2:, irg_ind].mean()*2260))

# Merge the x values and corresponding flux data
x_combined = np.concatenate([ -  np.flip(x[:-100]) ,x])
flux_combined = np.concatenate([FluxQ_fallow[100:, irg_ind] * 2260, FluxQ[:, irg_ind] * 2260])

plt.plot(
    x_combined, 
    flux_combined, 
    '--', c  = 'C1',
    label=f"Including advection,\nmean ET = {flux_combined.mean():.0f} W/m$^2$"
)

x_combined = np.concatenate([ -  np.flip(x[:-100]) ,x])
flux_combined = np.ones_like(x_combined) * FluxQ_uniform[nx-1, irg_ind]*2260

plt.plot(
    x_combined, 
    flux_combined, ':', c = 'C2',
    label=f"Continuous cultivation,\nmean ET = {flux_combined.mean():.0f} W/m$^2$"
)

plt.xlabel('x (m)', fontsize=12, fontweight='normal')
plt.ylabel('ET (W/m$^2$)', fontsize=12, fontweight='normal')
plt.title('Surface Flux (W/m$^2$)', fontsize=12)
ax.legend()#loc='center left', bbox_to_anchor=(1, 0.5))
ax.set_ylim(0, )
# (FluxQ[2:, irg_ind].sum()*dx/nx - FluxQ[nx:,  irg_ind].sum()*dx)/(FluxQ[nx:,  irg_ind].sum()*dx)


In [ ]:
# z = np.arange(params['zmin'] + params['dz'], params['zmax'] + params['dz'], params['dz'])
# x = np.arange(params['xmin'], params['xmax'] + params['dx'], params['dx'])

# Plot water vapor concentration and vertical flux
plt.figure(figsize = (8, 6))

plt.subplot(2, 1, 1)
plt.contourf(x, z, (Q_uniform.T), 25, cmap='Blues',  vmax = Q_c, vmin = Q_a)
plt.colorbar(label='Water vapor concentration \n (gm m$^{-3}$)')
plt.xlabel('x (m)', fontsize=12, fontweight='normal')
plt.ylabel('z (m)', fontsize=12, fontweight='normal')
plt.title('Water vapor concentration $q$ (gm m$^{-3}$)', fontsize=12)

plt.subplot(2, 1, 2)
plt.contourf(x, z[2:], FluxQ_uniform.T.round(4)[ 2:],  20, cmap='coolwarm', vmin = 0, vmax = 0.2)
plt.colorbar(label='Water vapor flux \n (gm m$^{-2}$ s$^{-1}$)') 
plt.xlabel('x (m)', fontsize=12, fontweight='normal')
plt.ylabel('z (m)', fontsize=12, fontweight='normal')
plt.title('Water vapor flux (gm m$^{-2}$ s$^{-1}$)', fontsize=12)
plt.tight_layout()



In [ ]:

z = np.arange(params['zmin'] + params['dz'], params['zmax'] + params['dz'], params['dz'])
x = np.arange(params['xmin'], params['xmax'] + params['dx'], params['dx'])

# Plot water vapor concentration and vertical flux
plt.figure(figsize = (8, 6))

plt.subplot(2, 1, 1)
plt.contourf(x, z, (Q_fallow.T), 25, cmap='Blues',  vmax = Q_f, vmin = Q_a)
plt.colorbar(label='Water vapor concentration \n (gm m$^{-3}$)')
plt.xlabel('x (m)', fontsize=12, fontweight='normal')
plt.ylabel('z (m)', fontsize=12, fontweight='normal')
plt.title('Water vapor concentration $q$ (gm m$^{-3}$)', fontsize=12)

plt.subplot(2, 1, 2)
plt.contourf(x[1:], z, FluxQ_fallow[1:].T.round(4),  20, cmap='coolwarm', vmin =0, vmax = 0.2)
plt.colorbar(label='Water vapor flux \n (gm m$^{-2}$ s$^{-1}$)') 
plt.xlabel('x (m)', fontsize=12, fontweight='normal')
plt.ylabel('z (m)', fontsize=12, fontweight='normal')
plt.title('Water vapor flux (gm m$^{-2}$ s$^{-1}$)', fontsize=12)
plt.tight_layout()


In [ ]:
# Plot water vapor concentration and vertical flux
plt.figure(figsize = (8, 6))

# Normalized concentration (if needed)
# Qd = (Q - Qa) / (Qs - Qa)

plt.subplot(2, 1, 1)
plt.contourf(x, z, (Q.T), 25, cmap='Blues',  vmax = 12, vmin = 8)
plt.colorbar(label='Water vapor concentration \n (gm m$^{-3}$)')
plt.xlabel('x (m)', fontsize=12, fontweight='normal')
plt.ylabel('z (m)', fontsize=12, fontweight='normal')
plt.title('Water vapor concentration $q$ (gm m$^{-3}$)', fontsize=12)

plt.subplot(2, 1, 2)
plt.contourf(x, z, FluxQ.T.round(4),  20, cmap='coolwarm', vmin = -1, vmax = 1)
plt.colorbar(label='Water vapor flux \n (gm m$^{-2}$ s$^{-1}$)') 
plt.xlabel('x (m)', fontsize=12, fontweight='normal')
plt.ylabel('z (m)', fontsize=12, fontweight='normal')
plt.title('Water vapor flux (gm m$^{-2}$ s$^{-1}$)', fontsize=12)

plt.tight_layout()
plt.show()


In [ ]:

# Normalized concentration
Q_d = (Q - Q_a) / (Q_c - Q_a)

plt.figure(3, figsize = (10,4))
plt.clf()
plt.contourf(x, z, Q_d.T, 20, vmax =1, cmap='Blues')
plt.xlabel(r'$\it{x}$ (m)', fontweight='bold', fontsize=10)
plt.ylabel(r'$\it{z}$ (m)', fontweight='bold', fontsize=10)
plt.colorbar(label='y=(Q-Q_a)/(Q_s-Q_a)')
plt.title('y=(Q-Q_a)/(Q_s-Q_a)')
